# Loading Libraries

In [ ]:
import torch
import torchvision
from torch.utils.data import ConcatDataset
import torchvision.transforms as transforms
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import time
import random
import numpy as np
import matplotlib.pyplot as plt
import torchvision.models as models

## Load and transform CIFAR10 dataset

In [ ]:
def create_data_loaders(batch_size, use_augmentation=False):
    # Standard CIFAR10 transforms
    transform = transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
    ])

    # Augmented CIFAR10 transforms
    train_transform_augmented = transforms.Compose([
        transforms.RandomHorizontalFlip(),
        transforms.RandomRotation(10),
        transforms.RandomAffine(0, shear=10, scale=(0.8, 1.2)),
        transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2),
        transforms.ToTensor(),
        transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
    ])

    # Load CIFAR10 dataset
    trainset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
    testset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)

    # Load augmented CIFAR10 dataset
    trainset_augmented = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=train_transform_augmented)

    # Combine datasets
    if use_augmentation:
        trainset_augmented = ConcatDataset([trainset, trainset_augmented])

    # Choose the appropriate trainset
    trainset_to_use = trainset_augmented if use_augmentation else trainset

    # Create data loaders
    trainloader = torch.utils.data.DataLoader(trainset_to_use, batch_size=batch_size, shuffle=True)
    testloader = torch.utils.data.DataLoader(testset, batch_size=batch_size, shuffle=False)

    return trainloader, testloader

In [ ]:
trainloader, testloader = create_data_loaders(64, use_augmentation=False)

# General Functions

## Visualization

In [ ]:
def display_filters(weights, filename):
    N = int(np.ceil(np.sqrt(weights.shape[0])))
    f, axarr = plt.subplots(N, N, figsize=(12, 12))
    scaled = (weights - weights.min()) / (weights.max() - weights.min())  # Scale the weights for better plotting

    p = 0
    for i in range(N):
        for j in range(N):
            # Empty plot white when out of kernels to display
            if p >= scaled.shape[0]:
                krnl = torch.ones((scaled.shape[2], scaled.shape[3], 3))
            else:
                if scaled.shape[1] == 1:
                    krnl = scaled[p, :, :, :].permute(1, 2, 0)
                    axarr[i, j].imshow(krnl, cmap="gray")
                elif scaled.shape[1] == 3:
                    krnl = scaled[p, :, :, :].permute(1, 2, 0)
                    axarr[i, j].imshow(krnl)
                else:
                    krnl = scaled[p, 0, :, :]
                    axarr[i, j].imshow(krnl, cmap="gray")
            axarr[i, j].axis("off")
            p += 1

    # Save the figure
    plt.savefig(filename)

    # Show the plot
    plt.show()

# Define the CustomCNN

In [ ]:
class CustomCNN(nn.Module):
    def __init__(self, use_batch_norm=False, use_dropout=False):
        super(CustomCNN, self).__init__()
        self.use_batch_norm = use_batch_norm
        self.use_dropout = use_dropout
        
        # Define the first set of convolutional layers
        self.conv1 = nn.Conv2d(3, 32, kernel_size=3, padding=1)
        self.bn1 = nn.BatchNorm2d(32) if self.use_batch_norm else nn.Identity()
        self.conv2 = nn.Conv2d(32, 32, kernel_size=3, padding=1)
        self.bn2 = nn.BatchNorm2d(32) if self.use_batch_norm else nn.Identity()
        self.pool1 = nn.MaxPool2d(2, 2)
        self.dropout1 = nn.Dropout(0.25) if self.use_dropout else nn.Identity()

        # Define the second set of convolutional layers
        self.conv3 = nn.Conv2d(32, 64, kernel_size=3, padding=1)
        self.bn3 = nn.BatchNorm2d(64) if self.use_batch_norm else nn.Identity()
        self.conv4 = nn.Conv2d(64, 64, kernel_size=3, padding=1)
        self.bn4 = nn.BatchNorm2d(64) if self.use_batch_norm else nn.Identity()
        self.pool2 = nn.MaxPool2d(2, 2)
        self.dropout2 = nn.Dropout(0.25) if self.use_dropout else nn.Identity()

        # Define the third set of convolutional layers
        self.conv5 = nn.Conv2d(64, 128, kernel_size=3, padding=1)
        self.bn5 = nn.BatchNorm2d(128) if self.use_batch_norm else nn.Identity()
        self.conv6 = nn.Conv2d(128, 128, kernel_size=3, padding=1)
        self.bn6 = nn.BatchNorm2d(128) if self.use_batch_norm else nn.Identity()
        self.pool3 = nn.MaxPool2d(2, 2)
        self.dropout3 = nn.Dropout(0.25) if self.use_dropout else nn.Identity()

        # Flatten layer
        self.flatten = nn.Flatten()

        # Fully connected layers
        self.fc1 = nn.Linear(128 * 4 * 4, 128)
        self.dropout4 = nn.Dropout(0.25) if self.use_dropout else nn.Identity()
        self.fc2 = nn.Linear(128, 10)
    
    def forward(self, x):
        # First set of layers
        x = self.conv1(x)
        if self.use_batch_norm:
            x = self.bn1(x)
        x = F.relu(x)
        x = self.conv2(x)
        if self.use_batch_norm:
            x = self.bn2(x)
        x = F.relu(x)
        x = self.pool1(x)
        if self.use_dropout:
            x = self.dropout1(x)

        # Second set of layers
        x = self.conv3(x)
        if self.use_batch_norm:
            x = self.bn3(x)
        x = F.relu(x)
        x = self.conv4(x)
        if self.use_batch_norm:
            x = self.bn4(x)
        x = F.relu(x)
        x = self.pool2(x)
        if self.use_dropout:
            x = self.dropout2(x)

        # Third set of layers
        x = self.conv5(x)
        if self.use_batch_norm:
            x = self.bn5(x)
        x = F.relu(x)
        x = self.conv6(x)
        if self.use_batch_norm:
            x = self.bn6(x)
        x = F.relu(x)
        x = self.pool3(x)
        if self.use_dropout:
            x = self.dropout3(x)

        # Flatten the output
        x = self.flatten(x)

        # Fully connected layers
        x = self.fc1(x)
        x = F.relu(x)
        if self.use_dropout:
            x = self.dropout4(x)
        x = self.fc2(x)

        return x

# Instantiate the CustomCNN model

In [ ]:
network = CustomCNN(use_batch_norm=True, use_dropout=True)

### Load custom_cnn_model_best_task2 model weights from Task 2

In [ ]:
model_path = "custom_cnn_model_best_task2.pth"
network.load_state_dict(torch.load(model_path, map_location=torch.device('cpu')))
network.eval()

### Apply the Visualization to Convolutional Layers of CustomCNN

In [ ]:
conv_layers = [network.conv1, network.conv2, network.conv3, network.conv4, network.conv5, network.conv6]

for i, conv_layer in enumerate(conv_layers):
    print(f"Filters of Conv Layer {i+1}:")
    filters = conv_layer.weight.data.clone().cpu()
    filename = f"conv_layer_{i+1}_filters.png"  # Filename for saving the figure
    display_filters(filters, filename)

## Creating an Extractor Model for the CustomCNN model

In [ ]:
class ActivationExtractor(nn.Module):
    def __init__(self, original_model):
        super(ActivationExtractor, self).__init__()
        # Copy layers from the original model up to the flatten layer
        self.features = nn.Sequential(
            original_model.conv1,
            original_model.bn1,
            original_model.conv2,
            original_model.bn2,
            original_model.pool1,
            original_model.dropout1,
            original_model.conv3,
            original_model.bn3,
            original_model.conv4,
            original_model.bn4,
            original_model.pool2,
            original_model.dropout2,
            original_model.conv5,
            original_model.bn5,
            original_model.conv6,
            original_model.bn6,
            original_model.pool3,
            original_model.dropout3,
            original_model.flatten
        )

    def forward(self, x):
        x = self.features(x)
        return x

In [ ]:
# Initialize the extractor model
extractor = ActivationExtractor(network)
# Ensure the extractor model is on the same device as the inputs
extractor.to('cuda' if torch.cuda.is_available() else 'cpu')

### Extracting Activations

In [ ]:
activations = []
for data in testloader:
    inputs, _ = data
    inputs = inputs.to('cuda' if torch.cuda.is_available() else 'cpu')
    # Detach the output from the computation graph before converting to numpy
    extracted_features = extractor(inputs).detach().cpu().numpy()
    activations.append(extracted_features)

activations = np.concatenate(activations)

# Check the shape of the extracted features
print("Shape of extracted activations:", activations.shape)

# Instantiate a pre-trained VGG16 model

In [ ]:
vgg16 = models.vgg16(pretrained=True)

# Set the model to evaluation mode
vgg16.eval()

### Apply the Visualization to Convolutional Layers of VGG16

In [ ]:
# Iterate over the features module of VGG16
for i, layer in enumerate(vgg16.features):
    if isinstance(layer, nn.Conv2d):
        print(f"Filters of Conv Layer {i}:")
        filters = layer.weight.data.clone().cpu()
        filename = f"vgg16_conv_layer_{i}_filters.png"  # Filename for saving the figure
        display_filters(filters, filename)